In [1]:
L = 300
N = 256
cutoff = 1e9
redDir = '/mnt/marvin2/bigsims/HighZ/HighZ_L300_N6144/halos_b0.25/z10.000/'
fileExt = 'z_14'
perKept = 0.1

In [2]:
import os
import h5py
import asdf
import numpy as np
from abacusnbody.analysis.tsc import tsc_parallel
from abacusnbody.data.read_abacus import read_asdf

In [4]:
haloDir = redDir + 'halo_info/'
haloDir = '/mnt/marvin2/bigsims/AbacusSummit/AbacusSummit_base_c000_ph0'+"{:02d}".format(24)+'/halos/z5.000/halo_info/'
haloFiles = np.array(os.listdir(haloDir))[np.char.startswith(os.listdir(haloDir), 'h')]

In [5]:
af = asdf.open(haloDir + haloFiles[1])
boxSize = af['header']['BoxSize']
partMass = af['header']['ParticleMassHMsun']

/home/kboone/miniconda3/envs/galbias/lib/python3.12/site-packages/asdf/_compression.py:46: AsdfWarning: Found more than one compressor for "blsc"
  warnings.warn(f'Found more than one compressor for "{label}"', AsdfWarning)


In [6]:
af['header']['BoxSize']

2000.0

In [5]:
pos = np.array([[],[],[]]).T
N_parts = np.array([])
for haloFile in haloFiles:
    af = asdf.open(haloDir + haloFile)
    pos = np.append(pos, af['data']['x_com'][:], axis = 0)
    N_parts = np.append(N_parts, af['data']['N'][:])
if np.abs((np.max(pos)-0.5)/0.5) > 0.01:
    raise Exception('Unexpected positional arguments')
sizeCrop = np.where(np.linalg.norm(pos * boxSize, ord = np.inf, axis = 1) <= L/2)[0]
N_parts = N_parts[sizeCrop]
pos = boxSize * pos[sizeCrop]

In [6]:
gala_pos = pos[np.where(N_parts * partMass >= cutoff)[0]] + L/2

In [7]:
with h5py.File('Data/Gala_'+fileExt+'.h5', 'w') as f:
    dset = f.create_dataset(
        'dataset',
        data=gala_pos.T.astype(np.float32),
        chunks=(1, len(gala_pos)),
        compression='gzip'
    )

In [8]:
scaleCrop = L/2

fieldDir = redDir + 'field_rv_A/'
haloDir = redDir + 'halo_rv_A/'

fieldFiles = np.array(os.listdir(fieldDir))[np.char.startswith(os.listdir(fieldDir), 'f')]
haloFiles = np.array(os.listdir(haloDir))[np.char.startswith(os.listdir(haloDir), 'h')]

In [9]:
allCrops = []

for fieldFile in fieldFiles:
    pos = np.array(read_asdf(fieldDir + fieldFile, load = ['pos'])['pos'])
    if np.max([np.min(np.abs(pos[:,0])), np.min(np.abs(pos[:,1])), np.min(np.abs(pos[:,2]))]) > scaleCrop:
        continue
        
    mask = np.random.rand(len(pos)) <= perKept
    pos = pos[mask]
    
    crop = pos[np.where(np.linalg.norm(pos, ord = np.inf, axis = 1) < scaleCrop)[0]]
    allCrops.append(crop)

for haloFile in haloFiles:
    pos = np.array(read_asdf(haloDir + haloFile, load = ['pos'])['pos'])
    if np.max([np.min(np.abs(pos[:,0])), np.min(np.abs(pos[:,1])), np.min(np.abs(pos[:,2]))]) > scaleCrop:
        continue

    mask = np.random.rand(len(pos)) <= perKept
    pos = pos[mask]
    
    crop = pos[np.where(np.linalg.norm(pos, ord = np.inf, axis = 1) < scaleCrop)[0]]
    allCrops.append(crop)

/home/kboone/miniconda3/envs/galbias/lib/python3.12/site-packages/asdf/_compression.py:46: AsdfWarning: Found more than one compressor for "blsc"
  warnings.warn(f'Found more than one compressor for "{label}"', AsdfWarning)


In [10]:
totalLen = 0
for i in np.arange(len(allCrops)):
    totalLen += len(allCrops[i])

In [11]:
cropsArr = np.empty((totalLen, 3))

startInd = 0
for i in np.arange(len(allCrops)):
    cropsArr[startInd:startInd + len(allCrops[i])] = allCrops[i]
    startInd += len(allCrops[i])

In [ ]:
cropsArr += L/2

In [12]:
with h5py.File('Data/DM_'+fileExt+'.h5', 'w') as f:
    dset = f.create_dataset(
        'dataset',
        data=cropsArr.T.astype(np.float32),
        chunks=(1, len(cropsArr)),
        compression='gzip'
    )

In [13]:
d_matter = tsc_parallel(cropsArr.astype(np.float32), N, L, nthread=32)
ave_matter = np.average(d_matter)
d_matter = (d_matter / np.average(d_matter)) - 1
d_matter = d_matter - np.average(d_matter)

np.save('Data/Density_DM_'+fileExt+'.npy', d_matter)